In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
#langchain tracking
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING"]=os.getenv("LANGSMITH_TRACING")


In [14]:
import streamlit as st
from langchain_groq import ChatGroq
from langchain.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

In [15]:

#setting up llm
llm=ChatGroq(model="llama-3.1-8b-instant",api_key=api_key)


In [16]:
llm.invoke("what is the capital of usa?")

AIMessage(content='The capital of the United States of America is Washington, D.C. (District of Columbia).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 42, 'total_tokens': 62, 'completion_time': 0.026666667, 'prompt_time': 0.002417301, 'queue_time': 0.052738519, 'total_time': 0.029083968}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'finish_reason': 'stop', 'logprobs': None}, id='run--73f63fe4-d3a1-4245-8d94-7f6b49e1f663-0', usage_metadata={'input_tokens': 42, 'output_tokens': 20, 'total_tokens': 62})

In [17]:

#chat prompt template
prompt = ChatPromptTemplate.from_template(
    """
    
    You are a helpful assistant. Answer the question based on the context  provided.
    <context>
     {context}
     <context>
     
     question: {question}
    
    """
)


In [18]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_huggingface import HuggingFaceEmbeddings
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
model_name = "Qwen3-Embedding-0.6B"
embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B"
)

print(embeddings.embed_query("What is the capital of India?"))

[-0.03699173405766487, -0.042218927294015884, -0.004668944515287876, -0.03709208965301514, 0.009943329729139805, -0.03257011994719505, 0.007992973551154137, -0.05514698103070259, -0.036013055592775345, -0.04366052895784378, 0.024823883548378944, 0.014198100194334984, 0.027803927659988403, -0.005250687710940838, -0.04299693554639816, 0.012150047346949577, 0.007986907847225666, 0.06713131815195084, 0.06604223698377609, -0.0463210754096508, 0.02618899941444397, -0.003587357932701707, 0.004651302471756935, -0.0010477089090272784, -0.018802586942911148, 0.052200257778167725, -0.06958480924367905, 0.09123539179563522, -0.011276008561253548, -0.050805892795324326, 0.10899815708398819, 0.07966753095388412, -0.039652079343795776, -0.017308715730905533, -0.030733909457921982, -0.007011269684880972, 0.014623090624809265, -0.01498865894973278, -0.03803127259016037, 0.035819072276353836, -0.0019021691987290978, -0.021076496690511703, -0.04588421806693077, 0.006499290466308594, -0.011821873486042023

In [8]:
def create_vector_embeddings():
     if "vectors" not in st.session_state:
          st.session_state.embeddings=OllamaEmbeddings(model="llama-3.1-8b")
          st.session_state.loader = PyPDFDirectoryLoader("/research papers")
          st.session_state.documents = st.session_state.loader.load()
          st.session_state.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
          st.session_state.docs = st.session_state.text_splitter.split_documents(st.session_state.documents)
          #created vector store db
          st.session_state.vectors = FAISS.from_documents(st.session_state.docs, st.session_state.embeddings)

user_prompt=st.text_input("Enter your question here")

if st.button("Documnet Embedding"):
     create_vector_embeddings()
     st.success("Document embeddings created successfully!\n Vectors are created and stored in session state.")
import time

if user_prompt:
     #documnet chain
     document_chain=create_stuff_documents_chain(llm=llm, prompt=prompt)
     #retriever
     retriver=st.session_state.vectors.as_retriever()
     #retrievel chain
     retrivel_chain=create_retrieval_chain(retriever=retriver, document_chain=document_chain)
     
     start=time.process_time()
     response=retrivel_chain.invoke({"question":user_prompt})
     print(f"response time: {time.process_time()-start}")
     
     st.write(response["answer"])
     
     with st.expander("Context"):
          for doc in response["context"]:
               st.write(doc.page_content)
               st.write("Source:", doc.metadata.get("source", "Unknown"))
               st.write("---")
     
    
          
          
                                                       
          
     

2025-07-07 10:05:25.873 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 10:05:25.875 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 10:05:25.876 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 10:05:25.877 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 10:05:25.878 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-07-07 10:05:25.878 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Streamlit requires raw Python (.py) files, not .ipynb.
For more information, please see https://docs.streamlit.io
